<a href="https://colab.research.google.com/github/lambroz/Jane-Street-Puzzles/blob/master/KnightMoves6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
import numpy
import numba
import itertools
import collections

In [ ]:
def generate_board(A, B, C):
    board = numpy.array([[A, B, B, C, C, C],
                         [A, B, B, C, C, C],
                         [A, A, B, B, C, C],
                         [A, A, B, B, C, C],
                         [A, A, A, B, B, C],
                         [A, A, A, B, B, C]])
    return board

In [ ]:
board = generate_board(1,2,3)

In [ ]:
board

array([[1, 2, 2, 3, 3, 3],
       [1, 2, 2, 3, 3, 3],
       [1, 1, 2, 2, 3, 3],
       [1, 1, 2, 2, 3, 3],
       [1, 1, 1, 2, 2, 3],
       [1, 1, 1, 2, 2, 3]])

In [ ]:
# If I rotate the board by 90 degrees clockwise, I can recycle the paths
numpy.rot90(board, k=3)

array([[1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 2, 2],
       [1, 1, 2, 2, 2, 2],
       [2, 2, 2, 2, 3, 3],
       [2, 2, 3, 3, 3, 3],
       [3, 3, 3, 3, 3, 3]])

In [ ]:
def rotate_element_90_cw(board, x, y):
    n = board.shape[0]
    new_x = n - 1 - y
    new_y = x
    return (new_x, new_y)

# Search for paths

In [ ]:
def knight_paths(board_size=6, max_path_length = 36, min_path_length = 0):

    def is_valid_move(x, y):
        return 0 <= x < board_size and 0 <= y < board_size

    def backtrack(x, y, path, max_path_length, min_path_length):

        if len(path) > max_path_length:
            return

        elif (x, y) == (board_size-1, board_size-1):
          if min_path_length <= len(path):
            paths.append(path)
          else:
            return

        else:
            knight_moves = [
                (-2, -1), (-2, 1), (-1, -2), (-1, 2),
                (1, -2), (1, 2), (2, -1), (2, 1)
            ]

            for dx, dy in knight_moves:
                next_x, next_y = x + dx, y + dy
                if is_valid_move(next_x, next_y) and (next_x, next_y) not in path:
                    backtrack(next_x, next_y, path + [(next_x, next_y)], max_path_length, min_path_length)

    paths = []

    backtrack(0, 0, [(0, 0)], max_path_length, min_path_length)

    return paths

In [ ]:
# Same function as above, just faster

@numba.njit
def is_valid_move(x, y, board_size):
    return 0 <= x < board_size and 0 <= y < board_size

@numba.njit
def backtrack(x, y, path, paths, visited, board_size, path_length, max_path_length, min_path_length):
    # Check if the path length exceeds the maximum allowed length
    if len(path) > max_path_length:
        return path_length

    # Check if we've reached the bottom-right corner
    if (x, y) == (board_size - 1, board_size - 1):
        # Store the path in the paths array
        if len(path) >= min_path_length:
          for i in range(len(path)):
              paths[path_length, i, 0] = path[i][0]
              paths[path_length, i, 1] = path[i][1]
          return path_length + 1  # Increment the count of good paths that have been found
        else:
          return path_length
    else:
        # Define the possible knight moves
        knight_moves = [
            (-2, -1), (-2, 1), (-1, -2), (-1, 2),
            (1, -2), (1, 2), (2, -1), (2, 1)
        ]
        for dx, dy in knight_moves:
            next_x, next_y = x + dx, y + dy
            if is_valid_move(next_x, next_y, board_size) and not visited[next_x, next_y]:
                visited[next_x, next_y] = True
                path.append((next_x, next_y))
                path_length = backtrack(next_x, next_y, path, paths, visited, board_size, path_length, max_path_length, min_path_length)
                path.pop()  # Backtrack
                visited[next_x, next_y] = False
        return path_length

@numba.njit
def knight_paths_njit(board_size=6, max_path_length=36, min_path_length=0):
    max_paths = 100_000_000  # Set a maximum number of paths
    max_path_storage = max_path_length  # Maximum path length that could be stored
    paths = numpy.empty((max_paths, max_path_storage, 2), dtype=numpy.int64)  # Preallocate space for paths
    visited = numpy.zeros((board_size, board_size), dtype=numpy.bool_)
    visited[0, 0] = True
    path = [(0, 0)]
    path_length = 0
    path_length = backtrack(0, 0, path, paths, visited, board_size, path_length, max_path_length, min_path_length)
    return paths[:path_length]  # Return only the valid paths, the remaining paths are empty

In [ ]:
BOARD_SIZE = 6

t0 = time.time()
#paths = knight_paths(board_size=BOARD_SIZE, max_path_length=12, min_path_length=8)
paths = knight_paths_njit(board_size=BOARD_SIZE, max_path_length=15, min_path_length=15)
t1 = time.time()

print(f'For a board {BOARD_SIZE}x{BOARD_SIZE}, number of paths found = {len(paths)}, in {t1-t0}s')

For a board 6x6, number of paths found = 629294, in 22.798808574676514s


# Test paths

In [ ]:
def score(path, board):

    # First element is the starting point
    x, y = path[0]
    score = board[x, y]
    previous_element = board[x, y]

    for x, y in path[1:]:
        current_element = board[x, y]
        if current_element == previous_element:
            score += current_element
        else:
            score *= current_element
        previous_element = current_element

    return score

In [ ]:
def chess_notation_to_matrix_postion(s):
  return 6 - int(s[1]), ord(s[0]) - 97

In [ ]:
def matrix_position_to_chess_notation(x, y):
    rank = str(6 - x)
    f = chr(y + 97)
    return f + rank

In [ ]:
GOAL = 2024

In [ ]:
def find_trip(test_triplet, paths):

  for triplet in itertools.permutations(test_triplet):

    A, B, C = triplet

    board = generate_board(A, B, C)
    rotated_board = numpy.rot90(board, k=3)

    trip_one = None
    trip_two = None

    for path in paths:

      if trip_one is None:
        if score(path, rotated_board) == GOAL:
          trip_one = [rotate_element_90_cw(board, x, y) for x, y in path] # Rotate back

      if trip_two is None:
        if score(path, board) == GOAL:
          trip_two = path.tolist()

      if trip_one is not None and trip_two is not None:
        print(f'Good triplet: {triplet}')
        print(f'Found trip 1: {trip_one}')
        print(f'Found trip 2: {trip_two}')
        answer = f'{A},{B},{C},' + ','.join([matrix_position_to_chess_notation(x, y) for x, y in trip_one]) + ',' + ','.join([matrix_position_to_chess_notation(x, y) for x, y in trip_two])
        print(answer)
        return answer

    print(f'No good triplets found for {triplet}')

In [ ]:
test_triplet = [1, 2, 3] # This is the smallest triplet, potential 'best_trip'
answer = find_trip(test_triplet, paths)

No good triplets found for (1, 2, 3)
Good triplet: (1, 3, 2)
Found trip 1: [(5, 0), (3, 1), (1, 0), (2, 2), (3, 0), (1, 1), (0, 3), (2, 4), (1, 2), (0, 4), (2, 3), (0, 2), (2, 1), (1, 3), (0, 5)]
Found trip 2: [[0, 0], [1, 2], [0, 4], [2, 3], [3, 1], [4, 3], [2, 4], [0, 5], [1, 3], [0, 1], [2, 0], [3, 2], [5, 3], [3, 4], [5, 5]]
1,3,2,a1,b3,a5,c4,a3,b5,d6,e4,c5,e6,d4,c6,b4,d5,f6,a6,c5,e6,d4,b3,d2,e4,f6,d5,b6,a4,c3,d1,e3,f1


# Validation

In [ ]:
def score_string(s):
  l = s.split(',')
  A, B, C = int(l[0]), int(l[1]), int(l[2])
  board = generate_board(A, B, C)

  trip_one = []
  trip_two = []

  fill_trip_one = True
  for position in l[3:]:

    if position == 'a6':
      fill_trip_one = False

    matrix_postion = chess_notation_to_matrix_postion(position)
    if fill_trip_one:
      trip_one.append(matrix_postion)
    else:
      trip_two.append(matrix_postion)

  print(f'Trip one: {trip_one}')
  print(f'Trip two: {trip_two}')
  print(f'Trip one score: {score(trip_one, board)}')
  print(f'Trip two score: {score(trip_two, board)}')

In [ ]:
test_input = '1,2,253,a1,b3,c5,d3,f4,d5,f6,a6,c5,a4,b2,c4,d2,f1'
score_string(test_input)

Trip one: [(5, 0), (3, 1), (1, 2), (3, 3), (2, 5), (1, 3), (0, 5)]
Trip two: [(0, 0), (1, 2), (2, 0), (4, 1), (2, 2), (4, 3), (5, 5)]
Trip one score: 2024
Trip two score: 2024


In [ ]:
score_string(answer)

Trip one: [(5, 0), (3, 1), (1, 0), (2, 2), (3, 0), (1, 1), (0, 3), (2, 4), (1, 2), (0, 4), (2, 3), (0, 2), (2, 1), (1, 3), (0, 5)]
Trip two: [(0, 0), (1, 2), (0, 4), (2, 3), (3, 1), (4, 3), (2, 4), (0, 5), (1, 3), (0, 1), (2, 0), (3, 2), (5, 3), (3, 4), (5, 5)]
Trip one score: 2024
Trip two score: 2024
